# Predicting the price of used cars

In a world where AI and basic code can help you automate systems, reflect data patterns, and even create meaningful web content, it promises to be a useful tool for companies of any size. 
Do you like to reminisce on memories with old friends? Most likely YES! Growing up, we all wanted to own a car someday(new or old) because it felt as a rite of passage.Buying a car especially a new one, don't come cheap.Due to the cost implication, majority tend to go for the second choice which is used car market. In the process of buying or selling that car, you can either pay too much or sell less than it's market value.

In this project we are going to predict the price of used cars using machine learning techniques. The dataset can be downloaded from Kaggle. link below >>>
https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho


In [3]:
#importing libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
sns.set_style('whitegrid')
sns.set_palette("GnBu_d")